In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# ===============================================================
# 1. IMPORT LIBRARIES
# ===============================================================
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")


In [2]:
# Users metadata
users = pd.read_csv("/kaggle/input/matchverse-2025-ai-matrimonial-challenge/users.csv")

# Interactions (positive interactions only)
interactions = pd.read_csv("/kaggle/input/matchverse-2025-ai-matrimonial-challenge/interactions.csv")

# Test / Submission file
recommendations = pd.read_csv("/kaggle/input/matchverse-2025-ai-matrimonial-challenge/recommendations.csv")

# Quick view
print(users.head())
print(interactions.head())
print(recommendations.head())


   Member_ID  Gender  Age Marital_Status   Sect             Caste  \
0          7    Male   27  Never Married  Sunni    Khan or Pathan   
1         10  Female   25  Never Married  Sunni  Sheikh or Shaikh   
2         30  Female   36  Never Married  Sunni             Other   
3         38    Male   33  Never Married  Sunni              Syed   
4         63    Male   34  Never Married  Sunni  Sheikh or Shaikh   

           State  
0          Bihar  
1          Bihar  
2          Delhi  
3  Uttar Pradesh  
4  Uttar Pradesh  
   Member_ID  Target_ID
0          7     407974
1          7         30
2          7        452
3          7        515
4          7       1014
       ID  Member_ID  top_100_profiles
0  137544     137544               NaN
1   16569      16569               NaN
2  439283     439283               NaN
3  278297     278297               NaN
4  121606     121606               NaN


In [4]:
# Encode categorical columns
cat_cols = ['Gender', 'Marital_Status', 'Sect', 'Caste', 'State']
le_dict = {}

for col in cat_cols:
    le = LabelEncoder()
    users[col] = le.fit_transform(users[col].astype(str))
    le_dict[col] = le

# Features to use for similarity
features = ['Gender','Age','Marital_Status','Sect','Caste','State']

# Standardize numeric features
scaler = StandardScaler()
users_scaled = users.copy()
users_scaled[features] = scaler.fit_transform(users_scaled[features])


NameError: name 'LabelEncoder' is not defined

In [1]:
# Using features to compute similarity
user_features_matrix = users_scaled[features].values
user_similarity = cosine_similarity(user_features_matrix)

# Convert to DataFrame for easy indexing
user_sim_df = pd.DataFrame(user_similarity, index=users['Member_ID'], columns=users['Member_ID'])


NameError: name 'users_scaled' is not defined

In [6]:
# Dictionary: Member_ID -> set of past interactions
interaction_dict = interactions.groupby('Member_ID')['Target_ID'].apply(set).to_dict()

# Ensure everyone has an entry
for member in users['Member_ID']:
    if member not in interaction_dict:
        interaction_dict[member] = set()


NameError: name 'interactions' is not defined

In [5]:
def recommend_profiles(member_id, top_n=100):
    # All users sorted by similarity
    sim_scores = user_sim_df.loc[member_id].sort_values(ascending=False)
    
    # Exclude self
    sim_scores = sim_scores.drop(member_id)
    
    # Exclude past interactions
    past = interaction_dict.get(member_id, set())
    sim_scores = sim_scores[~sim_scores.index.isin(past)]
    
    # Ensure opposite gender (0=Male, 1=Female) - adjust based on your encoding
    member_gender = users.loc[users['Member_ID']==member_id, 'Gender'].values[0]
    opposite_gender = 1 - member_gender
    candidates = users.loc[users['Gender']==opposite_gender, 'Member_ID']
    
    sim_scores = sim_scores[sim_scores.index.isin(candidates)]
    
    # Pick top N
    top_profiles = sim_scores.head(top_n).index.tolist()
    
    return ','.join(map(str, top_profiles))
